In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
from keras.layers import Conv1D, Activation, Flatten, Dense
from keras.models import Sequential

import pandas as pd
import librosa
import numpy as np

root_path=r'/content/drive/MyDrive/ravdess'

actors = ['Actor_01', 'Actor_02', 'Actor_03', 'Actor_04', 'Actor_05']

In [ ]:
features=[]
labels=[]
for actor in actors:
  file_names = os.listdir(os.path.join(root_path,actor))
  for i in range(len(file_names)):
    full_path = os.path.join(root_path, actor, file_names[i])
    splits= file_names[i].split('.')[0].split('-')
    label = splits[2]
    modality=splits[0]
    if modality=='01':
      raw_audio, sample_rate = librosa.load(full_path, sr=22050, duration=3)
      mfccs=librosa.feature.mfcc(y=raw_audio, sr=22050, n_mfcc=13)
      mfccs = np.mean(mfccs,axis=0)
      features.append(mfccs)
      labels.append(int(label))

features = np.asarray(features)
labels = np.asarray(labels)

features.shape, labels.shape

((300, 130), (300,))

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=7)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
from sklearn.preprocessing import LabelEncoder

lb = LabelEncoder()
y_train = np_utils.to_categorical(lb.fit_transform(y_train))
y_test = np_utils.to_categorical(lb.fit_transform(y_test))

x_train = np.expand_dims(X_train, axis=2)
x_test = np.expand_dims(X_test, axis=2)

In [ ]:
from keras.models import Model
from keras import Input

In [ ]:
inputs = Input(shape=(X_train.shape[1],1))
x=Conv1D(32, 5, padding='same')(inputs)
x=Conv1D(32, 5, padding='same')(x)
x=Activation('relu')(x)
x=Conv1D(64, 3)(x)
x=Conv1D(64, 3)(x)
x=Flatten()(x)
x=Dense(8)(x)
x=Activation('softmax')(x)

In [ ]:
model = Model(inputs=inputs, outputs=x)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=50, batch_size=64)